# Tutorial 1 - A simple pipeline to plot a signal

First we define a simple pipeline of two agents, of which one will generate a signal 
(in our case a *SineGeneratorAgent*) and the other one plots the signal on the 
dashboard (this is always a *MonitorAgent*).

We define a *SineGeneratorAgent* for which we have to override the 
functions `init_parameters()` & `agent_loop()` to define the new agent's behaviour.

*   `init_parameters()` is used to setup the input data stream and potentially other 
necessary parameters.
*   `agent_loop()` will be endlessly repeated until further notice. It will 
sample by sample extract the input data stream's content and push it to all agents 
connected to *SineGeneratorAgent*'s output channel by invoking `send_output()`.

The *MonitorAgent* is connected to the *SineGeneratorAgent*'s output channel and per 
default automatically plots the output. 

Each agent has an internal `current_state` which can be used as a switch to change the 
behaviour of the agent. The available states are listed
[here](https://github.com/bangxiangyong/agentMET4FOF/blob/a95b788544e8cce1e0bb757184da8c6447e96927/agentMET4FOF/agents.py#L78).

As soon as all agents are initialized and the connections are set up, the agent 
network is started by accordingly changing all agents' state simultaneously.

In [ ]:
# %load tutorial_1_generator_agent.py
from agentMET4FOF.agents import AgentMET4FOF, AgentNetwork, MonitorAgent
from agentMET4FOF.streams import SineGenerator


class SineGeneratorAgent(AgentMET4FOF):
    """An agent streaming a sine signal

    Takes samples from the :py:mod:`SineGenerator` and pushes them sample by sample
    to connected agents via its output channel.
    """
    _sine_stream: SineGenerator

    def init_parameters(self):
        """Initialize the input data

        Initialize the input data stream as an instance of the
        :py:mod:`SineGenerator` class
        """
        self._sine_stream = SineGenerator()

    def agent_loop(self):
        """Model the agent's behaviour

        On state *Running* the agent will extract sample by sample the input data
        streams content and push it via invoking :py:method:`AgentMET4FOF.send_output`.
        """
        if self.current_state == "Running":
            sine_data = self._sine_stream.next_sample()  # dictionary
            self.send_output(sine_data["x"])


def demonstrate_generator_agent_use():
    # Start agent network server.
    agent_network = AgentNetwork()

    # Initialize agents by adding them to the agent network.
    gen_agent = agent_network.add_agent(agentType=SineGeneratorAgent)
    monitor_agent = agent_network.add_agent(agentType=MonitorAgent)

    # Interconnect agents by either way:
    # 1) by agent network.bind_agents(source, target).
    agent_network.bind_agents(gen_agent, monitor_agent)

    # 2) by the agent.bind_output().
    gen_agent.bind_output(monitor_agent)

    # Set all agents' states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution
    return agent_network


if __name__ == "__main__":
    demonstrate_generator_agent_use()
